# Change the correct gobjaverse-lvis dataset from gobjaverse format to objaverse format.

#### Step 1: change the 2nd and 1st level folder.

In [ ]:
import os
import json
import shutil
import numpy as np

# Change the 2nd-level folders.
with open("annotations/280k_index_to_objaverse/gobjaverse_280k_index_to_objaverse.json", "r") as json_file:
    mapping = json.load(json_file)

root_directory = 'The Download path.'

for parent_folder_name in os.listdir(root_directory):
    parent_folder_path = os.path.join(root_directory, parent_folder_name)
    
    if os.path.isdir(parent_folder_path):
        for folder_name in os.listdir(parent_folder_path):
            folder_path = os.path.join(parent_folder_path, folder_name)
            
            if os.path.isdir(folder_path):
                new_folder_name = mapping.get(folder_path[14:], folder_name).split('/')[-1].split('.')[0]
                new_folder_path = os.path.join(parent_folder_path, new_folder_name)
                os.rename(folder_path, new_folder_path)

# Change the 1st-level folders.
folders = os.listdir(root_directory)

for folder in folders:
    if os.path.isdir(os.path.join(root_directory, folder)):
        original_number = int(folder)
        new_name = f"000-{original_number:03d}"
        new_path = os.path.join(root_directory, new_name)
        os.rename(os.path.join(root_directory, folder), new_path)

#### Step 2: 
mkdir the pose and rgb
move .jpg and .json. and remove number_filepath  

In [ ]:
def mkdir_pose_and_rgb(root_directory):
    for first_level in os.listdir(root_directory):
        first_level_path = os.path.join(root_directory, first_level)
        if os.path.isdir(first_level_path):
            for second_level in os.listdir(first_level_path):
                second_level_path = os.path.join(first_level_path, second_level)
                if os.path.isdir(second_level_path):
                    os.makedirs(os.path.join(second_level_path, 'pose'), exist_ok=True)
                    os.makedirs(os.path.join(second_level_path, 'rgb'), exist_ok=True)



def move_files(directory):
    # Iterate through all files in the directory
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # Move .json files to the 'pose' folder
        if item.endswith('.json'):
            target_json_path = os.path.join(directory, '../pose')
            shutil.move(item_path, target_json_path)
        # Move .png files to the 'rgb' folder
        elif item.endswith('.png'):
            target_json_path = os.path.join(directory, '../rgb')
            shutil.move(item_path, target_json_path)

def move_and_delete(root_directory):
    # Iterate through the first-level directories in the root directory
    for first_level in os.listdir(root_directory):
        first_level_path = os.path.join(root_directory, first_level)
        if os.path.isdir(first_level_path):
            # Iterate through the second-level directories in the first-level directory
            for second_level in os.listdir(first_level_path):
                second_level_path = os.path.join(first_level_path, second_level)
                for j in range(40):
                    number_filename = f"{j:05}"
                    number_filepath = os.path.join(second_level_path, number_filename)
                    move_files(number_filepath)
                    shutil.rmtree(number_filepath)

mkdir_pose_and_rgb(root_directory)
move_and_delete(root_directory)


#### Step 3:  5 numbers -> 3 numbers.

In [ ]:
def change_number(current_directory):
    # Get the names of all folders in the current directory
    jsons_path = os.path.join(current_directory, 'pose')
    pngs_path = os.path.join(current_directory, 'rgb')
    jsons = os.listdir(jsons_path)
    pngs = os.listdir(pngs_path)

    for json in jsons:
        # Extract the numeric part from the filename, e.g., '00031' -> '31'
        file_number = json.split('.')[0][2:]
        png = json[:-4] + 'png'

        # Rename the files
        new_json = f"{file_number}.json"
        new_png = f"{file_number}.png"

        # Create new file paths
        new_json_path = os.path.join(jsons_path, new_json)
        old_json_path = os.path.join(jsons_path, json)

        new_png_path = os.path.join(pngs_path, new_png)
        old_png_path = os.path.join(pngs_path, png)

        # Rename the files
        os.rename(old_json_path, new_json_path)
        os.rename(old_png_path, new_png_path)

# Get the list of first-level folders
first_folders = os.listdir(root_directory)

# Iterate through each first-level folder
for first_folder in first_folders:
    first_folder_path = os.path.join(root_directory, first_folder)
    second_folders = os.listdir(first_folder_path)
    for second_folder in second_folders:
        second_folder_path = os.path.join(first_folder_path, second_folder)
        change_number(second_folder_path)

#### Step 4: .json to .txt

In [ ]:
error_paths = []

def json_to_txt(json_path_wo_json):
    try:
        with open(json_path_wo_json + '.json', 'r') as json_file:
            json_content = json.load(json_file)

        camera_matrix = np.eye(4)
        camera_matrix[:3, 0] = np.array(json_content['x'])
        camera_matrix[:3, 1] = np.array(json_content['y'])
        camera_matrix[:3, 2] = np.array(json_content['z'])
        camera_matrix[:3, 3] = np.array(json_content['origin'])

        pose_text = ' '.join(map(str, camera_matrix.flatten()))

        with open(json_path_wo_json + '.txt', 'w') as txt_file:
            txt_file.write(pose_text)
        if os.path.exists(json_path_wo_json + '.json'):
            os.remove(json_path_wo_json + '.json')

    except Exception as e:
        print(f"Error processing file {json_path_wo_json}: {str(e)}")
        error_paths.append(json_path_wo_json)


for first_level_dir in os.listdir(root_directory):
    first_level_path = os.path.join(root_directory, first_level_dir)
    if os.path.isdir(first_level_path):
        
        for second_level_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, second_level_dir, 'pose')
            
            for third_level_dir in os.listdir(second_level_path):
                third_level_path = os.path.join(second_level_path, third_level_dir).split('.')[0]
                json_to_txt(third_level_path)

with open('error_log.json', 'w') as error_file:
    json.dump(error_paths, error_file)


#### Step 5: train.json and val.json

In [ ]:
# Initialize an empty list to store all directory paths
directory_list = []

# Iterate through the first-level directories
for first_level_dir in os.listdir(root_directory):
    first_level_path = os.path.join(root_directory, first_level_dir)
    
    # Check if it is a directory
    if os.path.isdir(first_level_path):
        
        # Iterate through the second-level directories
        for second_level_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, second_level_dir)
            
            # Check if it is a directory
            if os.path.isdir(second_level_path):
                
                # Append the path to the list
                directory_list.append(os.path.join(first_level_dir, second_level_dir))

# Save the list to a JSON file
with open(os.path.join(root_directory, 'train.json'), 'w') as json_file:
    json.dump(directory_list, json_file)

Note that in this way, all objects are for training. In our settint, we only make objects in '000-159' are for validation and remaining is for training. You can download them in the hugging face.

In [ ]:
from huggingface_hub import hf_hub_download

# train.json: 000-000 -> 000-158
hf_hub_download(repo_id="alexzyqi/GPT4Point", filename='train.json', repo_type="dataset", local_dir=root_directory)

# val.json: 000-159
hf_hub_download(repo_id="alexzyqi/GPT4Point", filename='val.json', repo_type="dataset", local_dir=root_directory)